## Exercise 4

Submitted by: Miguel Yapan\
ID Number: 205501\
Date: 03-08-2022

In [1]:
#Import pertinent libraries
import numpy as np

#We define the class MarkovChain taken from our module.
class MarkovChain:
    def __init__(self, matrix, states=None):
        #the following lines of code verify that the matrix is column stochastic by adding the entries in each column.
        for i in range(0,len(matrix[0])):
            if sum(matrix[:,0]) != 1:
                raise ValueError('Matrix is not column stochastic.')
        self.matrix = matrix
        #the following lines of code set a the default [0 1 2 ... N-1] for the states.
        if states == None:
            self.states = [i for i in range(0,len(matrix))]
        else:
            self.states = states
        #the following line creates a dictionary of states to their indexes in the matrix.
        self.map = {self.states[i]:i for i in range(0,len(matrix))}
            
    def nextstate(self, state):
        #nextstate is a method that guesses the next state by passing the probabilities in the column of the initial state to the np.random.multinomial function.
        index = self.map[state]
        probability = self.matrix[:, index]
        draws = np.random.multinomial(1, probability) 
        stateindex = np.argmax(draws)
        return self.states[stateindex]
    
    def walk(self, state, N):
        #walk lists down N states from your input to the next N-1 states.
        #walk is basically nextstate iterated multiple times.
        currentstate = [state]
        for i in range(0, N-1):
            index = self.map[currentstate[i]]
            probability = self.matrix[:, index]
            draws = np.random.multinomial(1, probability)
            nextstateindex = np.argmax(draws)
            currentstate.append(self.states[nextstateindex])
        return currentstate
   
    def path(self, initstate, endstate):
        #path lists down states until your specified endstate.
        #the code is similar to walk, but it instead checks if the current state matches the end state.
        currentstate = [initstate]
        i=0
        while currentstate[len(currentstate)-1] != endstate: 
            index = self.map[currentstate[i]]
            probability = self.matrix[:, index]
            draws = np.random.multinomial(1, probability)
            nextstateindex = np.argmax(draws)
            currentstate.append(self.states[nextstateindex])
            i += 1
        return currentstate
        
    def sdvector(self, initstate, N):
        #sdvector shows the state distribution vector of the (N+1)th state given an initial state.   
        #note that a the probability matrix is a square matrix with row and column equal to x, x is a positive integer.
        #in order to take the (N+1)th state distribution vector, we need to build a vector similar to [[0],[0],...,[1],...,[0]], which we'll call the initial vector or initvector. 
        #in the initvector, there there are x-1 zeroes and 1 one that is placed in the index of our input initial state.
        #the following lines of code build this vector.
        index = self.map[initstate]
        initlist = []
        for i in range(0, len(self.matrix)):
            initlist.append([0])
        initlist[index] = [1]
        initvector = np.array(initlist)
        #the following just converts our array to a matrix and then does the multiplication stated previously.
        matrix = np.matrix(self.matrix)
        return (matrix**(N))*initvector

### Assignment 4A

We first pass the given transition matrix `A` and labels `states` to our MarkovChain class as `good_defective`. To find out the probability of the 20th item being defective, we just pass `'defective'` and `19` to the `sdvector` method created from the MarkovChain class. We pass `19` because we want to find out the distribution in the 20th state. After running the code, we see that the probability of the 20th item being defective is 0.14240233 or 14.24%.

Alternatively, we can use `walk` multiple times and compute for the probability of the last element in the list being defective. The second block runs this solution. The probability it outputs varies but it usually ranges from 12% to 16%, which is around the same ballpark as our first solution. Raising the amount of iterations might help, but it will also cause it to run slower.

In [2]:
#Assignment 4A

A = np.array([[0.99, 0.12],[0.01, 0.88]])
states = ['good', 'defective']
good_defective = MarkovChain(A, states)

print(good_defective.sdvector('defective', 19))

[[0.85759767]
 [0.14240233]]


In [4]:
#Assignment 4A

final_list = []
i=0
while i <= 1000:
    data = good_defective.walk('defective', 20)
    final_list.append(data[-1]) 
    i += 1
probability = final_list.count('defective') / len(final_list) 
print(probability)
#The result here varies but I usually get a probability of 13% to 14%

0.14585414585414586


### Assignment 4B

Just like Assignment 4A, we first assign the given transition matrix to `A` and the labels to `states` to a MarkovChain class `bus`. oSince we want to know the probability of each state after a long period of time, we just pass `sdvector` to each state for a huge number. If the resulting vectors are the same, then we have our result. If not, then we check for a higher number. We first set `N=100`. The resulting probability of early arrivals, on-time arrivals, and late arrivals is 22.5%, 35%, and 42.5%, respectively, for all states. This is enough, but just for another sanity check, we verify this result for `N=1000`.

Alternatively, we can apply the second solution from 4A to 4B by getting the state after 1000 iterations of each initial state times using `walk`. We first set an empty list `probability` that will contain the state distribution vector from each initial state. Just like our answer in 4A, the probabilities vary but are more or less in the same range. I got 20% to 24% for early arrivals, 33% to 35% for on-time arrivals, and 40% to 44% for late arrivals.

In [5]:
#Assignment 4B

A = np.array([[0.5, 0.2, 0.1],[0.4,0.5,0.2],[0.1,0.3,0.7]])
states = ['early arrival', 'on-time arrival', 'late arrival']
bus = MarkovChain(A, states)

print(bus.sdvector('early arrival', 100))
print(bus.sdvector('on-time arrival', 100))
print(bus.sdvector('late arrival', 100))

print(bus.sdvector('early arrival', 1000))
print(bus.sdvector('on-time arrival', 1000))
print(bus.sdvector('late arrival', 1000))

[[0.225]
 [0.35 ]
 [0.425]]
[[0.225]
 [0.35 ]
 [0.425]]
[[0.225]
 [0.35 ]
 [0.425]]
[[0.225]
 [0.35 ]
 [0.425]]
[[0.225]
 [0.35 ]
 [0.425]]
[[0.225]
 [0.35 ]
 [0.425]]


In [6]:
#Assignment 4B

probability = []
for state in states:
    final_list = []
    i=0
    while i <= 1000:
        data = bus.walk(state, 1000)
        final_list.append(data[-1]) 
        i += 1
    probability.append([final_list.count('early arrival') / len(final_list), final_list.count('on-time arrival') / len(final_list), final_list.count('late arrival') / len(final_list)])

print(probability)

[[0.23176823176823177, 0.33666333666333664, 0.43156843156843155], [0.22077922077922077, 0.3386613386613387, 0.4405594405594406], [0.22577422577422576, 0.34965034965034963, 0.4245754245754246]]
